# Семинар №4

ФИО: Килимчук Иван Вячеславович

вуз: МГУ 

факультет: механико-математический

курс: 3

кафедра (если есть): МаТИС

научный руководитель (если есть): Иванов

#### Семинар подготовлен с целью ознакомить слушателей с процедурой подготовки данных для алгоритмов машнинного обучения на примере задачи оценки цены недвижимости. Описание данных можно найти в data_description.txt

### 1. Считывание данных (1 балл)

Считайте данные из файлов train.csv и test.csv в массивы:
trainX (содержит признаки обучающего множества)
trainY (содержит правильные ответа для обучающего множества)
testX (содержит признаки для тестового множества)

Первый столбец содержит порядковый номер объекта, поэтому его рекомендуется сразу удалить

In [16]:
import pandas as pd
import numpy as np

train_data = pd.read_csv('train.csv')
train_data = train_data.iloc[:, 1:]

trainX = train_data.drop('SalePrice', axis=1)
trainY = train_data['SalePrice']

testX = pd.read_csv('test.csv')
testX = testX.iloc[:, 1:]
print(testX.dtypes)

print(trainX.shape, trainY.shape, testX.shape)

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
Length: 79, dtype: object
(1460, 79) (1460,) (1459, 79)


In [4]:
print(trainX.iloc[0])

MSSubClass           60
MSZoning             RL
LotFrontage        65.0
LotArea            8450
Street             Pave
                  ...  
MiscVal               0
MoSold                2
YrSold             2008
SaleType             WD
SaleCondition    Normal
Name: 0, Length: 79, dtype: object


### 2. Предварительная обработка данных

Как правило "реальные" данные содержат пропущенные значения и прочие нечисловые признаки, поэтому прежде чем запустить методы fit и  predict модели, необходимо сделать все признаки числовыми.

#### 2.1 Обработка пропущенных значений (2 балла)

Изучите раздел документации https://scikit-learn.org/stable/modules/impute.html#impute и параметры классов SimpleImputer и MissingIndicator

In [3]:
from sklearn.impute import MissingIndicator, SimpleImputer

In [4]:
indicator = MissingIndicator()
train_mask_missing_values_only = indicator.fit_transform(trainX)
test_mask_missing_values_only = indicator.fit_transform(testX)
print(np.sum(train_mask_missing_values_only))
print(np.sum(test_mask_missing_values_only))

7829
7878


В данных были обнаружены пропущенные значение. 
При помощи класса MissingIndicator устраните пропущенные значения, использовав самый часто встречаемый элемент.

In [15]:
imputer = SimpleImputer(strategy='most_frequent')

trainX_without_missing = imputer.fit_transform(trainX)
testX_without_missing = imputer.fit_transform(testX)

AttributeError: 'numpy.ndarray' object has no attribute 'dtypes'

In [9]:
indicator = MissingIndicator()
train_mask_missing_values_only = indicator.fit_transform(trainX_without_missing)
test_mask_missing_values_only = indicator.fit_transform(testX_without_missing)
print(np.sum(train_mask_missing_values_only))
print(np.sum(test_mask_missing_values_only))

0
0


Пропущенные значения удалены.

In [10]:
print(trainX_without_missing[0])

[60 'RL' 65.0 8450 'Pave' 'Grvl' 'Reg' 'Lvl' 'AllPub' 'Inside' 'Gtl'
 'CollgCr' 'Norm' 'Norm' '1Fam' '2Story' 7 5 2003 2003 'Gable' 'CompShg'
 'VinylSd' 'VinylSd' 'BrkFace' 196.0 'Gd' 'TA' 'PConc' 'Gd' 'TA' 'No'
 'GLQ' 706 'Unf' 0 150 856 'GasA' 'Ex' 'Y' 'SBrkr' 856 854 0 1710 1 0 2 1
 3 1 'Gd' 8 'Typ' 0 'Gd' 'Attchd' 2003.0 'RFn' 2 548 'TA' 'TA' 'Y' 0 61 0
 0 0 0 'Gd' 'MnPrv' 'Shed' 0 2 2008 'WD' 'Normal']


#### 2.2  Обработка категориальных значений (2 балла)

Изучите раздел документации https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features

При помощи класса OrdinalEncoder удалите категориальные признаки.

In [164]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()

trainX_without_missing = pd.DataFrame(trainX_without_missing)
categorical_features = trainX_without_missing.select_dtypes(include=['object']).columns.tolist()
trainX_without_missing[categorical_features] = encoder.fit_transform(trainX_without_missing[categorical_features])

testX_without_missing = pd.DataFrame(testX_without_missing)
categorical_features = testX_without_missing.select_dtypes(include=['object']).columns.tolist()
testX_without_missing[categorical_features] = encoder.fit_transform(testX_without_missing[categorical_features])

trainX_without_missing_and_cat = trainX_without_missing
testX_without_missing_and_cat = testX_without_missing

In [165]:
print(trainX_without_missing_and_cat[0])

0       5.0
1       0.0
2       5.0
3       6.0
4       5.0
       ... 
1455    5.0
1456    0.0
1457    6.0
1458    0.0
1459    0.0
Name: 0, Length: 1460, dtype: float64


Теперь данные готовы для обучения моделей

Опишите достоинства и недостатки данного метода обработки категориальных признаков

ваш ответ


### 3. Разделение данных на обучение и валидацию (1 балл)

Разделите обучающее множество на обучение(75%) и валидацию(25%), воспользовавшись функцией train_test_split с random_state=42

In [166]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(trainX_without_missing_and_cat, trainY, test_size=0.25, random_state=42)

In [167]:
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(1095, 79) (365, 79) (1095,) (365,)


### 4. Обучение моделей (4 балла)

Данные готовы для обучения алгоритмов машинного обучения. В качестве базовой можеди возьмём линейную регрессию, меткрика качества - mean_squared_log_error

In [168]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_log_error

In [170]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)
y_pred[y_pred < 0.0] = 0.0

print(mean_squared_log_error(y_val, y_pred))

0.38236907560850375


Текущую модель можно существенно улучшить. Добейтесь на валидации ошибки меньше 0.03 без каких-либо ограничений на алгоритмы

In [180]:
from sklearn.ensemble import GradientBoostingRegressor

gbm = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gbm.fit(X_train, y_train)
y_pred = gbm.predict(X_val)
y_pred[y_pred < 0.0] = 0.0

print(mean_squared_log_error(y_val, y_pred))

0.0189020235035036


### 5. Способы улучшения качества модели

1. Применить другую стратегию обработки пропущенных значений из scikit-learn (1 балл + 1 балл, если получили прирост качества для алгоритма с ошибкой меньше 0.03 )

2. Добавить бинарные признаки отсутствия/присутствия значения для столбцов, где есть пропущенные значения (1 балл)

3. Реализовать самостоятельно одну из стратегий обработки пропущенных значений из семинара (начиная с kNN) (2 балла)

4. Применить стратегию OneHotEncoder  для обработки категориальных признаков (2 балла)

5. Применить стратегию агрегированния данных для обработки категориальных признаков (2 балла)

Реализуйте не менее двух идей из предложенных выше